In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from refinegems.classes.gapfill import KEGGapFiller
from refinegems.utility.io import load_model

/Users/brune/miniconda3/envs/sprg/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


In [3]:
mpath = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JCSC1435.xml'

model = load_model(mpath,'libsbml')

In [4]:
gff2 = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JCSC1435_RefSeq.gff'
missing_genes = get_missing_genes(gff2,load_model('/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JCSC1435.xml','libsbml'))
missing_genes

# @TODO : some od the genes do not have an old locus tag -> what to do?

NameError: name 'get_missing_genes' is not defined

In [3]:
from refinegems.classes.gapfill import GeneGapFiller
from refinegems.utility.io import load_model

modelpath = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435.xml'
model = load_model(modelpath,'libsbml')
cmodel = load_model(modelpath,'cobra')

gffpath = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435_RefSeq.gff'

gf2 = GeneGapFiller()
gf2_missing_genes = gf2.get_missing_genes(gffpath,model)
# ncbiprotein | locus_tag | ec-code

tfasta = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435_proteins_genome.fasta'
spdb = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/swissprot.dmnd'
# gf2_missing_genes
spmap = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/uniprot_table.tsv'
kwargs = {'outdir':'/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435',
          'sens':'more-sensitive',
          'cov':90.0,
          't':4,
          'pid':90.0}

mapped_res = gf2.get_missing_reacs(model=cmodel,
                                   missing_genes=gf2_missing_genes,
                                   fasta=tfasta, 
                                   dmnd_db=spdb,
                                   swissprot_map=spmap,
                                   **kwargs)

/Users/brune/miniconda3/envs/sprg/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


Running in debugging mode.


In [7]:
mapped_res[1]

,ec-code,ncbiprotein,id,equation,reference,is_transport,via,add_to_GPR
0,5.6.2.2,[WP_011274363.1],MNXR115632,1 MNXM12437@MNXD1 + 1 MNXM40333@MNXD1 + 1 MNXM...,metacycR:5.99.1.3-RXN,None,MetaNetX,None
1,5.6.2.2,[WP_011274363.1],MNXR172894,1 MNXM1100221@MNXD1 + 1 MNXM40333@MNXD1 + 1 MN...,sabiorkR:15120,None,MetaNetX,None
2,5.6.2.2,[WP_011274363.1],MNXR172895,1 MNXM1100221@MNXD1 + 1 MNXM735047@MNXD1 + 1 M...,sabiorkR:15121,None,MetaNetX,None
3,5.6.2.2,[WP_011274363.1],MNXR172896,1 MNXM1100223@MNXD1 + 1 MNXM40333@MNXD1 + 1 MN...,sabiorkR:15122,None,MetaNetX,None


### SwissProt Setup

In [6]:
# @TODO
# SwissProt Download
# (mappings + sequences can be downloaded from DB)
# two tables: 
#   1.: TSV with UniprotID, BRENDA and EC -7.7MB (26.07.2024)
#   2.: FASTA with sequences ~280MB (26.07.2024)
# more infos:
#   - API generation possible for automatic download
#   - maybe write a small script to download stuff and shorten to actual EC/BRENDA mappings

# current_release or complete? differences?
swissprot_api = 'https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.fasta.gz'
swissprot_mapping_api = 'https://rest.uniprot.org/uniprotkb/stream?compressed=true&fields=accession%2Cxref_brenda%2Cec%2Csequence&format=tsv&query=%28*%29+AND+%28reviewed%3Atrue%29'

# use Python code e.g. requests or
# use a bash script for this? (OS-independancy might become a problem...)

def download_swissprot_for_gapfill():
    pass
    # download FASTA
    # download mapping
    # (optionally) contruct DIAMOND DB